In [59]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack, csr_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score
import scipy.sparse as sparse
import string
import gc

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import preprocessing 

from sklearn.model_selection import GridSearchCV
import seaborn as sns
%matplotlib inline
sns.set(style="darkgrid")

In [2]:
df=pd.read_csv('trainwHD.csv',encoding='utf-8')

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53447 entries, 0 to 53515
Data columns (total 5 columns):
Webpage_id    53447 non-null float64
Tag           53447 non-null object
web_text      53447 non-null object
url_length    53447 non-null float64
url_text      53447 non-null object
dtypes: float64(2), object(3)
memory usage: 4.9+ MB


In [34]:
del df['Unnamed: 0']
df.drop(df[df['Webpage_id'].isnull()].index.values,inplace=True)
df.drop(df[df['web_text'].isnull() & df['url_length'].isnull() & df['url_text'].isnull()].index.values,inplace=True)
df['web_text'].fillna('a',inplace=True)
df['url_text'].fillna('a',inplace=True)
df['url_length'].fillna(0,inplace=True)

In [45]:
df.isnull().sum()

Webpage_id    0
Tag           0
web_text      0
url_length    0
url_text      0
dtype: int64

In [120]:
df_test=pd.read_csv('testwHD.csv',encoding='utf-8')

In [162]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25776 entries, 0 to 25775
Data columns (total 5 columns):
index         25776 non-null int64
Webpage_id    25776 non-null float64
web_text      25776 non-null object
url_length    25776 non-null float64
url_text      25776 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 1007.0+ KB


SyntaxError: invalid syntax (<ipython-input-161-e3e0e8ca73f8>, line 1)

In [122]:
del df_test['Unnamed: 0']
df_test.drop(df_test[df_test['Webpage_id'].isnull()].index.values,inplace=True)
df_test.drop(df_test[df_test['web_text'].isnull() & df_test['url_length'].isnull() & df_test['url_text'].isnull()].index.values,inplace=True)

In [163]:
df_test.isnull().sum()

index         0
Webpage_id    0
web_text      0
url_length    0
url_text      0
dtype: int64

In [149]:
dddd=list(pd.read_csv('./data/test.xls')['Webpage_id'].unique())

In [154]:
df_test.drop(df_test[df_test['Webpage_id'].apply(lambda x: False if x in dddd else True)].index.values,inplace=True)

In [ ]:
df_test['web_text'].fillna('a',inplace=True)
df_test['url_text'].fillna('a',inplace=True)
df_test['url_length'].fillna(0,inplace=True)

In [57]:
df.reset_index(inplace=True)
df_test.reset_index(inplace=True)

In [60]:
vect_word = TfidfVectorizer(max_features=1000, analyzer='word', stop_words='english', ngram_range=(1,2), dtype=np.float32) 
vect_word.fit(df.url_text) 
tr_urlvect = vect_word.transform(df.url_text) 

In [61]:
vect_web = TfidfVectorizer(max_features=5000, analyzer='word', stop_words='english', ngram_range=(1,2), dtype=np.float32) 
vect_web.fit(df.web_text) 
tr_webvect = vect_web.transform(df.web_text) 

In [62]:
Y = df['Tag']
train = df[['url_length']]

In [63]:
std = RobustScaler()
train = pd.DataFrame(std.fit_transform(train)).set_index(train.index)

NameError: name 'tr_vect' is not defined

In [64]:
train_features = hstack([tr_urlvect,tr_webvect, csr_matrix(train.loc[train.index,])], 'csr')

In [71]:
train_features.data

array([0.69884858, 0.71526964, 0.02032314, ..., 0.02157022, 0.01436913,
       0.02072332])

In [72]:
model1 = XGBClassifier()
model1.fit(train_features,Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [73]:
y_pred_train_xgb = model1.predict(train_features)

C:\Users\Hardika\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [74]:
accuracy = accuracy_score(Y, y_pred_train_xgb)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
f1_score(Y, y_pred_train_xgb, average='weighted')  

Accuracy: 94.57%


0.9455834454726196

In [76]:
model2 = LogisticRegression()
model2.fit(train_features,Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [79]:
y_pred_train_lr = model2.predict(train_features)

In [80]:
accuracy = accuracy_score(Y, y_pred_train_lr)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
f1_score(Y, y_pred_train_lr, average='weighted')  

Accuracy: 96.45%


0.9644040979984709

In [164]:
vect_word.fit(df_test.url_text) 
test_urlvect = vect_word.transform(df_test.url_text) 

vect_web.fit(df_test.web_text) 
test_webvect = vect_web.transform(df_test.web_text) 

test = df_test[['url_length']]

test = pd.DataFrame(std.fit_transform(test)).set_index(test.index)
test_features = hstack([test_urlvect,test_webvect, csr_matrix(test.loc[test.index,])], 'csr')
y_pred_test_lr = model2.predict(test_features)
lrI = df_test['Webpage_id']
lrT = pd.DataFrame(y_pred_test_lr)

FRlr= pd.DataFrame()
FRlr['Webpage_id']=lrI
FRlr['Tag']=lrT



PermissionError: [Errno 13] Permission denied: './Submission/Summary_SubmissionFLRwHD.csv'

In [165]:
FRlr.to_csv('./Submission/Summary_SubmissionFLRwHD.csv')